In [24]:
# !brew install gcc@8

Updating Homebrew...
==> Downloading https://homebrew.bintray.com/bottles-portable-ruby/portable-ruby--2.6.3.mavericks.bottle.tar.gz
######################################################################## 100.0%                80.7%
==> Pouring portable-ruby--2.6.3.mavericks.bottle.tar.gz
==> Auto-updated Homebrew!
Updated 1 tap (homebrew/core).
==> New Formulae
alp                                      lazydocker
anime-downloader                         lerna
antibody                                 libevhtp
appium                                   libsigc++@2
arduino-cli                              libsixel
asyncplusplus                            libspng
atasm                                    libtensorflow@1
aws-cdk                                  libvncserver
aws-google-auth                          lightgbm
base91                                   lizard
bazelisk                                 llvm@8
benthos                                  mariadb@10.3
berglas               

cclive                     libguess                   postgres-xc
cherokee                   liblacewing                protobuf@3.1
compose2kube               libopkele                  scala@2.10
csup                       llvm@3.9                   skipfish
ctunnel                    llvm@5                     smlnj
dsd                        lysp                       supersonic
engine_pkcs11              mongodb                    swig@3.04
erlang@18                  mongodb@3.0                taisei
frag_find                  mongodb@3.2                thc-pptp-bruter
freetds@0.91               mongodb@3.4                tlsdate
ftimes                     mongodb@3.6                tn5250
guile@2.0                  mysql-cluster              varnish@4
httest                     nordugrid-arc              voms
irods                      packetbeat                 whirr
konoha                     pbrt                       xar-mackyle
kontena                    percona-server-mongo

In [27]:
# conda install -c conda-forge xgboost

Solving environment: done

## Package Plan ##

  environment location: /Users/mikhaillenko/anaconda3

  added / updated specs:
    - xgboost


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         143 KB  conda-forge
    conda-4.7.12               |           py36_0         3.0 MB  conda-forge
    libxgboost-0.90            |       h6de7cb9_2         1.4 MB  conda-forge
    llvm-openmp-9.0.0          |       h40edb58_0         266 KB  conda-forge
    py-xgboost-0.90            |   py36h6de7cb9_2          73 KB  conda-forge
    scikit-learn-0.21.3        |   py36h27c97d8_0         6.0 MB
    xgboost-0.90               |   py36h6de7cb9_2          11 KB  conda-forge
    ------------------------------------------------------------
                                     

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor
import xgboost as xg

/Users/mikhaillenko/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/mikhaillenko/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/mikhaillenko/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
# Reading in TRAIN (df) and TEST (fd) data
df = pd.read_csv('./data/df_select_features.csv')
fd = pd.read_csv('./data/fd_select_features.csv')
# To facilitate Kaggle submissions
submission_index = list(range(1461,2920))
# For referncing feature names
cols = list(fd.columns.values)

In [3]:
X = df.drop(columns=['SalePrice'])
y = df['SalePrice']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=42)

# Decision Tree Regression
    - Bagged Decision Trees Regressor
    - Random Forests Regressor
    - Extremely Random Trees Regressor

### Bagged Decision Tree Regressors

In [5]:
dtree = DecisionTreeRegressor()

param_grid = {'max_depth':[None,5,10,15,20],
              'min_samples_split':[2,4,6,8],
              'max_features':[None,5,10,15],
              'max_leaf_nodes':[None,5,10,15]}

gs_dtree = GridSearchCV(dtree, param_grid)

gs_dtree.fit(X_train, y_train)

/Users/mikhaillenko/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=DecisionTreeRegressor(criterion='mse', max_depth=None,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort=False, random_state=None,
                                             splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [None, 5, 10, 15, 20],
                         'max_features': [None, 5, 10, 15],
                         'max_leaf_nodes': [None, 5, 10, 15],
                        

In [6]:
# Generating predictions based on best model evaluated by GridSearch
dtree_preds = (gs_dtree.best_estimator_).predict(X_test)

print(f'(DTree) Mean cross validation score: {round(gs_dtree.best_score_, 4)}')
print(f'(DTree) Root mean squared log error: {round(((y_test-dtree_preds)**2).mean()**(1/2),4)}')
print()
print(f'(DTree) Best parameters, GridSearch: {gs_dtree.best_params_}')

(DTree) Mean cross validation score: 0.7573
(DTree) Root mean squared log error: 37823.6513

(DTree) Best parameters, GridSearch: {'max_depth': 10, 'max_features': 5, 'max_leaf_nodes': None, 'min_samples_split': 6}


In [7]:
# Bootstrapping and bagging trees

bag_df = pd.DataFrame(index=X_test.index)

for i in range(1000):
    # Random sampling with replacement
    X_sample = X_train.sample(n=X_train.shape[0],
                              replace=True)
    # Ensuring our random samples align with their label
    y_sample = y[X_sample.index]
    
    # Model configured according to GridSearch best_params_
    tree = DecisionTreeRegressor(max_depth=10,
                                 max_features=15,
                                 max_leaf_nodes=None,
                                 min_samples_split=8)
    # Fitting models
    tree.fit(X_sample, y_sample)
    
    # Appending predictions to DataFrame
    bag_df[f'Tree {i}'] = tree.predict(X_test)
    
bag_df.shape

(362, 1000)

In [8]:
bag_df['SalePrice'] = bag_df.mean(axis='columns')

RMSLE = ((np.log(y_test)-np.log(bag_df['SalePrice']))**2).mean()**(1/2)
print(f'Root mean squared logarithmic error: {round(RMSLE,4)}')
print(f'Mean cross validation score: {cross_val_score(dtree, X_train, y_train).mean()}')

Root mean squared logarithmic error: 0.1225
Mean cross validation score: 0.7376808293029665


/Users/mikhaillenko/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [9]:
cross_val_score(dtree, X, y)

/Users/mikhaillenko/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


array([0.7630766 , 0.6514854 , 0.75775347])

In [10]:
# Creating DataFrame for submission
tree_bag = pd.DataFrame(index=fd.index)

for i in range(1000):
    X_sample = X.sample(n=X.shape[0],
                         replace=True)
    y_sample = y[X_sample.index]
    
    tree = DecisionTreeRegressor(max_depth=10,
                                 max_features=15,
                                 max_leaf_nodes=None,
                                 min_samples_split=8)
    tree.fit(X_sample,y_sample)
    
    tree_bag[f'Tree {i}'] = tree.predict(fd)

In [11]:
tree_bag['SalePrice'] = tree_bag.mean(axis='columns')

In [12]:
# Creating submission DataFrame
tree_bag_submission = pd.DataFrame(columns=['Id','SalePrice'])
tree_bag_submission['Id'] = submission_index
tree_bag_submission['SalePrice'] = tree_bag['SalePrice']

# Exporting to .csv
tree_bag_submission.to_csv('./data/tree_bag.csv',index=False)

#### Kaggle score:
    - RMSLE: 0.1834

### Random Forest Regressor

In [13]:
# Instantiating Random Forrest Regressor
rf = RandomForestRegressor(random_state=42)

param_grid = {'n_estimators':[200,300,400],
              'max_depth':[None,15],
              'min_samples_split':[2,3,4],
              'max_features':[3,4,5],
              'max_leaf_nodes':[None,15]}

gs_ranfo = GridSearchCV(rf, param_grid, verbose=1)

gs_ranfo.fit(X_train, y_train)

/Users/mikhaillenko/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 108 candidates, totalling 324 fits


[Parallel(n_jobs=1)]: Done 324 out of 324 | elapsed:  2.1min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=42,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [None, 15], 'ma

In [14]:
rf_preds = (gs_ranfo.best_estimator_).predict(X_test)

print(f'(RandF) Mean cross validation score: {round(gs_ranfo.best_score_,4)}')
print(f'(RandF) Root mean squared log error: {round(((np.log(y_test)-np.log(rf_preds))**2).mean()**(1/2),4)}')

print()
print(f'Best parameters: {gs_ranfo.best_params_}')

(RandF) Mean cross validation score: 0.8717
(RandF) Root mean squared log error: 0.1177

Best parameters: {'max_depth': 15, 'max_features': 5, 'max_leaf_nodes': None, 'min_samples_split': 3, 'n_estimators': 200}


In [15]:
# Instantiating model according to GridSearch parameters
rf = RandomForestRegressor(max_depth=15,
                           max_features=5,
                           max_leaf_nodes=None,
                           min_samples_split=3,
                           n_estimators=200)

# Fitting model to entire training set for submission
rf.fit(X,y)

# Generating predictions for submission
rf_preds = rf.predict(fd)

# Creating DataFrame for submission
rf_df = pd.DataFrame(columns=['Id','SalePrice'])
rf_df['Id'] = submission_index
rf_df['SalePrice'] = rf_preds

In [16]:
rf_df.to_csv('./data/rand_for.csv', index=False)

#### Kaggle score:
    - RMSLE: 0.1727

### Extemely Random Trees Regressor

In [17]:
et = ExtraTreesRegressor(random_state=42)

param_grid = {'n_estimators':[200, 300, 400],
              'max_depth':[None,15],
              'min_samples_split':[2,3,4],
              'max_features':[3,4,5],
              'max_leaf_nodes':[None,15]}

gs_extra = GridSearchCV(et, param_grid, verbose=1)

gs_extra.fit(X_train, y_train)

/Users/mikhaillenko/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 108 candidates, totalling 324 fits


[Parallel(n_jobs=1)]: Done 324 out of 324 | elapsed:  1.9min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=ExtraTreesRegressor(bootstrap=False, criterion='mse',
                                           max_depth=None, max_features='auto',
                                           max_leaf_nodes=None,
                                           min_impurity_decrease=0.0,
                                           min_impurity_split=None,
                                           min_samples_leaf=1,
                                           min_samples_split=2,
                                           min_weight_fraction_leaf=0.0,
                                           n_estimators='warn', n_jobs=None,
                                           oob_score=False, random_state=42,
                                           verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [None, 15], 'max_features': [3, 4, 5],
                         'max_leaf_nodes':

In [18]:
et_preds = (gs_extra.best_estimator_).predict(X_test)

print(f'(Extra) Mean cross validation score: {round(gs_extra.best_score_,4)}')
print(f'(Extra) Root mean squared log error: {round(((np.log(y_test)-np.log(et_preds))**2).mean()**(1/2),4)}')
print()
print(f'(Extra) Best parameters: {gs_extra.best_params_}')

(Extra) Mean cross validation score: 0.8635
(Extra) Root mean squared log error: 0.1219

(Extra) Best parameters: {'max_depth': 15, 'max_features': 5, 'max_leaf_nodes': None, 'min_samples_split': 2, 'n_estimators': 300}


In [19]:
# Instantiating model according to GridSearch parameters
et = ExtraTreesRegressor(max_depth=15,
                         max_features=5,
                         max_leaf_nodes=None,
                         min_samples_split=2,
                         n_estimators=300)

# Fitting model to entire training set for submission
et.fit(X,y)

# Generating predictions for submission
et_preds = et.predict(fd)

# Creating DataFrame for submission
et_df = pd.DataFrame(columns=['Id','SalePrice'])
et_df['Id'] = submission_index
et_df['SalePrice'] = et_preds

In [20]:
et_df.to_csv('./data/extra_trees.csv', index=False)

#### Kaggle score:
    - RMSLE: 0.1644

### Adaptive Boosting

In [30]:
ada = AdaBoostRegressor(base_estimator=DecisionTreeRegressor())
ada_params = {'n_estimators':[40,50,60],
              'learning_rate':[1.0,0.9,0.8],
              'base_estimator__max_depth':[None, 15],
              'base_estimator__max_features':[3,4,5],
              'base_estimator__min_samples_split':[2,3,4]
             }
gs = GridSearchCV(ada, param_grid=ada_params, cv=5)
gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse',
                                                                              max_depth=None,
                                                                              max_features=None,
                                                                              max_leaf_nodes=None,
                                                                              min_impurity_decrease=0.0,
                                                                              min_impurity_split=None,
                                                                              min_samples_leaf=1,
                                                                              min_samples_split=2,
                                                                              min_weight_fraction_leaf=0.0,
                                                  

In [31]:
ada_preds = (gs.best_estimator_).predict(X_test)

print(f'(AdaB) Mean cross validation score: {round(gs.best_score_,4)}')
print(f'(AdaB) Root mean squared log error: {round(((np.log(y_test)-np.log(ada_preds))**2).mean()**(1/2),4)}')
print()
print(f'(AdaB) Best parameters: {gs.best_params_}')

(AdaB) Mean cross validation score: 0.8701
(AdaB) Root mean squared log error: 0.1194

(AdaB) Best parameters: {'base_estimator__max_depth': None, 'base_estimator__max_features': 4, 'base_estimator__min_samples_split': 4, 'learning_rate': 0.8, 'n_estimators': 50}


In [33]:
# Instantiating model according to GridSearch parameters
ada = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(max_depth=None, 
                                                             max_features=4, 
                                                             min_samples_split=4),
                        n_estimators=50,
                        learning_rate=0.8)
# Fitting model to entire training set for submission
ada.fit(X,y)

# Generating predictions for submission
ada_preds = ada.predict(fd)

# Creating DataFrame for submission
ada_df = pd.DataFrame(columns=['Id','SalePrice'])
ada_df['Id'] = submission_index
ada_df['SalePrice'] = ada_preds

In [35]:
# Exporting to .csv
ada_df.to_csv('./data/ada.csv', index=False)

#### Kaggle score:
    - RMSLE: 0.17932

### Gradient Boosting

In [39]:
grad_b = GradientBoostingRegressor(loss='huber')
grad_params = {'learning_rate':[0.05,0.1,0.15],
               'n_estimators': [75,100,125,200],
               'subsample':[0.9, 1.0],
               'min_samples_split':[2,3,4],
               'min_samples_leaf':[1,2,3],
               'max_depth':[2,3,4,5],
               'max_features':['auto','sqrt',5]}
gs = GridSearchCV(grad_b, param_grid=grad_params, cv=5, verbose=2)
gs.fit(X_train, y_train)

Fitting 5 folds for each of 2592 candidates, totalling 12960 fits
[CV] learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=75, subsample=0.9 
[CV]  learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=75, subsample=0.9, total=   0.2s
[CV] learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=75, subsample=0.9 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=75, subsample=0.9, total=   0.2s
[CV] learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=75, subsample=0.9 
[CV]  learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=75, subsample=0.9, total=   0.1s
[CV] learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=75, subsample=0.9 
[CV]  learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=75, subsample=0.9, total=   0.1s
[CV] learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=75, subsample=0.9 
[CV]  learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=75, subsample=0.9, total=   0.1s
[CV] learning_rate=0.05, max_d

[CV]  learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200, subsample=0.9, total=   2.0s
[CV] learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200, subsample=0.9 
[CV]  learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200, subsample=0.9, total=   0.8s
[CV] learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200, subsample=0.9 
[CV]  learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200, subsample=0.9, total=   0.4s
[CV] learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200, subsample=0.9 
[CV]  learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200, subsample=0.9, total=   0.5s
[CV] learning_rate=0.05

[CV]  learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=3, n_estimators=125, subsample=0.9, total=   0.3s
[CV] learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=3, n_estimators=125, subsample=0.9 
[CV]  learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=3, n_estimators=125, subsample=0.9, total=   0.3s
[CV] learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=3, n_estimators=125, subsample=0.9 
[CV]  learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=3, n_estimators=125, subsample=0.9, total=   0.3s
[CV] learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=3, n_estimators=125, subsample=0.9 
[CV]  learning_rate=0.05, max_depth=2, max_features=auto, min_samples_leaf=1, min_samples_split=3, n_estimators=125, subsample=0.9, total=   0.3s
[CV] learning_rate=0.05

...

[CV]  learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=2, n_estimators=200, subsample=1.0, total=   0.6s
[CV] learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=3, n_estimators=75, subsample=0.9 
[CV]  learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=3, n_estimators=75, subsample=0.9, total=   0.3s
[CV] learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=3, n_estimators=75, subsample=0.9 
[CV]  learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=3, n_estimators=75, subsample=0.9, total=   0.3s
[CV] learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=3, n_estimators=75, subsample=0.9 
[CV]  learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=3, n_estimators=75, subsample=0.9, total=   0.3s
[CV] learning_rate=0.15, max_depth=5, max_features

[CV]  learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=3, n_estimators=200, subsample=0.9, total=   0.7s
[CV] learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=3, n_estimators=200, subsample=0.9 
[CV]  learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=3, n_estimators=200, subsample=0.9, total=   0.7s
[CV] learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=3, n_estimators=200, subsample=0.9 
[CV]  learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=3, n_estimators=200, subsample=0.9, total=   0.7s
[CV] learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=3, n_estimators=200, subsample=0.9 
[CV]  learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=3, n_estimators=200, subsample=0.9, total=   0.7s
[CV] learning_rate=0.15, max_depth=5, max_fe

[CV]  learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=4, n_estimators=125, subsample=0.9, total=   0.5s
[CV] learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=4, n_estimators=125, subsample=0.9 
[CV]  learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=4, n_estimators=125, subsample=0.9, total=   0.5s
[CV] learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=4, n_estimators=125, subsample=0.9 
[CV]  learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=4, n_estimators=125, subsample=0.9, total=   0.5s
[CV] learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=4, n_estimators=125, subsample=0.9 
[CV]  learning_rate=0.15, max_depth=5, max_features=5, min_samples_leaf=3, min_samples_split=4, n_estimators=125, subsample=0.9, total=   0.5s
[CV] learning_rate=0.15, max_depth=5, max_fe

[Parallel(n_jobs=1)]: Done 12960 out of 12960 | elapsed: 148.3min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=GradientBoostingRegressor(alpha=0.9,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='huber', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=100,
                                                 n_i...
                             

In [43]:
grad_preds = (gs.best_estimator_).predict(X_test)

print(f'(GradB) Mean cross validation score: {round(gs.best_score_,4)}')
print(f'(GradB) Root mean squared log error: {round(((np.log(y_test)-np.log(grad_preds))**2).mean()**(1/2),4)}')
print()
print(f'(GradB) Best parameters: {gs.best_params_}')

(GradB) Mean cross validation score: 0.887
(GradB) Root mean squared log error: 0.1207

(GradB) Best parameters: {'learning_rate': 0.15, 'max_depth': 5, 'max_features': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'subsample': 0.9}


In [47]:
# Instantiating model according to GridSearch parameters
grad = GradientBoostingRegressor(loss='huber',
                                 max_depth=5, 
                                 max_features=5, 
                                 min_samples_split=2,
                                 min_samples_leaf=1,
                                 n_estimators=100,
                                 learning_rate=0.15,
                                 subsample=0.9)
# Fitting model to entire training set for submission
grad.fit(X,y)

# Generating predictions for submission
grad_preds = grad.predict(fd)

# Creating DataFrame for submission
grad_df = pd.DataFrame(columns=['Id','SalePrice'])
grad_df['Id'] = submission_index
grad_df['SalePrice'] = grad_preds

In [50]:
# Exporting to .csv
grad_df.to_csv('./data/grad.csv',index=False)

#### Kaggle score:
    - RMSLE: 0.17932

### XG Boost Regressor

In [53]:
# Instantiating XG Boost model
xg_boost = xg.XGBRegressor(objective='reg:linear',
                           booster='gbtree',
                           n_estimators=100,
                           n_jobs=4)
# Defining parameter grid for GridSearchCV
xg_params = {'max_depth':[3,4,5,6],
             'learning_rate':[0.05,0.1,0.15],
             'gamma':[0,0.1,0.5,1.0],
             'reg_alpha':[0,0.1,0.5,1.0],
             'reg_lambda':[1.0,1.25,1.5],
             'subsample':[0.5,0.9,1.0]}
# Instantiating GridSearchCV
gs = GridSearchCV(xg_boost,
                  param_grid=xg_params,
                  cv=5,
                  verbose=1,
                  n_jobs=4)
gs.fit(X_train, y_train)

Fitting 5 folds for each of 1728 candidates, totalling 8640 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    8.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   26.6s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  2.2min
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:  3.3min
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed:  4.6min
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed:  6.2min
[Parallel(n_jobs=4)]: Done 3192 tasks      | elapsed:  8.0min
[Parallel(n_jobs=4)]: Done 4042 tasks      | elapsed: 10.1min
[Parallel(n_jobs=4)]: Done 4992 tasks      | elapsed: 12.6min
[Parallel(n_jobs=4)]: Done 6042 tasks      | elapsed: 15.1min
[Parallel(n_jobs=4)]: Done 7192 tasks      | elapsed: 18.1min
[Parallel(n_jobs=4)]: Done 8442 tasks      | elapsed: 33.4min
[Parallel(n_jobs=4)]: Done 8640 out of 8640 | elapsed: 34.0min finished
/Users/mikhaillenko/anaconda3/lib/python3.6/site-pa

[13:42:41] WARNING: /usr/local/miniconda/conda-bld/xgboost_1566327371504/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=100, n_jobs=4, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='warn', n_jobs=4,
             param_grid={'gamma': [0, 0.1, 0.5, 1.0],
                         'learning_rate': [0.05, 0.1, 0.15],
          

In [54]:
xg_preds = (gs.best_estimator_).predict(X_test)

print(f'(GradB) Mean cross validation score: {round(gs.best_score_,4)}')
print(f'(GradB) Root mean squared log error: {round(((np.log(y_test)-np.log(xg_preds))**2).mean()**(1/2),4)}')
print()
print(f'(GradB) Best parameters: {gs.best_params_}')

(GradB) Mean cross validation score: 0.8773
(GradB) Root mean squared log error: 0.1142

(GradB) Best parameters: {'gamma': 0, 'learning_rate': 0.05, 'max_depth': 5, 'reg_alpha': 0, 'reg_lambda': 1.5, 'subsample': 0.5}


In [79]:
# Instantiating XGBoost according to GridSearch parameters
xgb = xg.XGBRegressor(max_depth=5,
                      learning_rate=0.05,
                      n_estimators=100,
                      objective='reg:linear',
                      n_jobs=4,
                      gamma=0,
                      reg_alpha=0,
                      reg_lambda=1.5,
                      subsample=0.5)

# Fitting model to entire training set for submission
xgb.fit(X,y)

# Generating predictions for submission
xgb_preds = xgb.predict(fd)

# Creating DataFrame for submission
xgb_df = pd.DataFrame(columns=['Id','SalePrice'])
xgb_df['Id'] = submission_index
xgb_df['SalePrice'] = xgb_preds

[15:16:54] WARNING: /usr/local/miniconda/conda-bld/xgboost_1566327371504/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [87]:
xgb_df.to_csv('./data/xgb.csv',index=False)

#### Kaggle score:
    - RMSLE: 0.1926

### XGBoost Random Forest Regressor

In [89]:
# Instantiating XGBoost RF Regressor
xgrf = xg.XGBRFRegressor(n_jobs=4,
                         objective='reg:squarederror',
                         n_estimators=100,
                         num_parallel_tree=100,
                         eta=1)
# Defining parameter grid for GridSearchCV
xgrf_params = {'max_depth':[3,4,5,6],
               'gamma':[0,0.1],
               'reg_alpha':[0,0.1],
               'reg_lambda':[1.0,1.25,1.5],
               'subsample':[0.4,0.6,0.8],
               'colsample_bynode':[0.2,0.25,0.5],
               'colsample_bytree':[0.8,1],
               'colsample_bylevel':[0.8,1]}
# Instantiating GridSearchCV
gs = GridSearchCV(xgrf,
                  param_grid=xgrf_params,
                  cv=5,
                  n_jobs=5,
                  verbose=1)
gs.fit(X_train, y_train)

Fitting 5 folds for each of 1728 candidates, totalling 8640 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    8.0s
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:   19.2s
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed:   39.5s
[Parallel(n_jobs=5)]: Done 790 tasks      | elapsed:  1.1min
[Parallel(n_jobs=5)]: Done 1240 tasks      | elapsed:  1.8min
[Parallel(n_jobs=5)]: Done 1790 tasks      | elapsed:  2.7min
[Parallel(n_jobs=5)]: Done 2440 tasks      | elapsed:  3.7min
[Parallel(n_jobs=5)]: Done 3190 tasks      | elapsed:  4.8min
[Parallel(n_jobs=5)]: Done 4040 tasks      | elapsed:  6.3min
[Parallel(n_jobs=5)]: Done 4990 tasks      | elapsed:  7.7min
[Parallel(n_jobs=5)]: Done 6040 tasks      | elapsed:  9.3min
[Parallel(n_jobs=5)]: Done 7190 tasks      | elapsed: 11.0min
[Parallel(n_jobs=5)]: Done 8440 tasks      | elapsed: 13.3min
[Parallel(n_jobs=5)]: Done 8640 out of 8640 | elapsed: 13.7min finished
/Users/mikhaillenko/anaconda3/lib/python3.6/site-pa

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=XGBRFRegressor(base_score=0.5, colsample_bylevel=1,
                                      colsample_bynode=0.8, colsample_bytree=1,
                                      eta=1, gamma=0, learning_rate=1,
                                      max_delta_step=0, max_depth=3,
                                      min_child_weight=1, missing=None,
                                      n_estimators=100, n_jobs=4, nthread=None,
                                      num_parallel_tree=100,
                                      objective='reg:squarederror',
                                      random_state=0, reg_...
                                      silent=None, subsample=0.8, verbosity=1),
             iid='warn', n_jobs=5,
             param_grid={'colsample_bylevel': [0.8, 1],
                         'colsample_bynode': [0.2, 0.25, 0.5],
                         'colsample_bytree': [0.8, 1], 'gamma': [0, 0.1],
  

In [90]:
xgrf_preds = (gs.best_estimator_).predict(X_test)

print(f'(GradB) Mean cross validation score: {round(gs.best_score_,4)}')
print(f'(GradB) Root mean squared log error: {round(((np.log(y_test)-np.log(xgrf_preds))**2).mean()**(1/2),4)}')
print()
print(f'(GradB) Best parameters: {gs.best_params_}')

(GradB) Mean cross validation score: 0.7991
(GradB) Root mean squared log error: 0.143

(GradB) Best parameters: {'colsample_bylevel': 0.8, 'colsample_bynode': 0.5, 'colsample_bytree': 1, 'gamma': 0, 'max_depth': 6, 'reg_alpha': 0.1, 'reg_lambda': 1.0, 'subsample': 0.8}


In [91]:
# Instantiating XGBoost Random Forest Regressor
# according to GridSearch parameters
xgrf = xg.XGBRFRegressor(colsample_bylevel=0.8,
                         colsample_bynode =0.5,
                         colsample_bytree =1.0,
                         gamma = 0,
                         max_depth = 6,
                         reg_alpha = 0.1,
                         reg_lambda = 1.0,
                         subsample = 0.8)
# Fitting model to entire training set for submission
xgrf.fit(X,y)

# Generating predictions for submission
xgrf_preds = xgrf.predict(fd)

# Creating DataFrame for submission
xgrf_df = pd.DataFrame(columns=['Id','SalePrice'])
xgrf_df['Id'] = submission_index
xgrf_df['SalePrice'] = xgrf_preds

/Users/mikhaillenko/anaconda3/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/mikhaillenko/anaconda3/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[18:54:09] WARNING: /usr/local/miniconda/conda-bld/xgboost_1566327371504/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [94]:
# Exporting to .csv
xgrf_df.to_csv('./data/xgrf.csv',index=False)

#### Kaggle score:
    - RMSLE: 0.1810